Import all the necessary lib

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

#### Web_driver Initialization

In [2]:
chrome_options = Options()
service = Service('C:\\chromedriver\\chromedriver.exe')

In [3]:
driver = webdriver.Chrome(service=service, options=chrome_options)

In [4]:
driver.get('https://stock-pictures.netlify.app')

In [5]:
soup = bs(driver.page_source, 'html.parser')

#### Scraping the Data and forming Data Frame 

In [6]:
data = []

for sp in soup.find_all('div', class_ = 'container'):
    if('gif' not in sp.find('img').get('src')):
        
        link = sp.find('img').get('src')
        tags = list(set(sp.find('div',class_ = 'tags').text[7:].split(' ')))     
        likes    = int(sp.find('div',class_ = 'likes-comments').find_all('span')[0].text[:-6])
        comments = int(sp.find('div',class_ = 'likes-comments').find_all('span')[1].text[:-9])
        
        data.append([link, ' '.join(tags), likes, comments])

In [7]:
data = pd.DataFrame(data, columns = ['Link', 'Tags', 'Likes', 'Comments'])
data

,Link,Tags,Likes,Comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Atmosphere, Sky Sky, Clouds, Blue",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Hummingbird Ornithology,",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainfall, Rainbow, Subtropical",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Japan, Sakura Blossoms, Road, Cherry",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Flower, Marguerite, Cape Plant",39,15
...,...,...,...,...
1898,https://cdn.pixabay.com/photo/2018/02/16/22/08...,"Gull Flying, Seagull, Bird, Wings,",627,143
1899,https://cdn.pixabay.com/photo/2022/03/22/16/00...,"Sea, Beach Cyprus, Island, Paphos,",5,0
1900,https://cdn.pixabay.com/photo/2013/02/21/19/10...,"Sea, Clouds, Boat Horizon, Cloudy,",867,146
1901,https://cdn.pixabay.com/photo/2022/03/23/16/22...,"Spring Blossom, Flowers, Cherry",11,8


In [8]:
data.to_csv('ImageScraping.csv')

#### Downloading the Images and Storing in a Folder

In [9]:
import requests

In [23]:
loc =[]

for i in data['Link']:
    res = requests.get(i)
    
    fd = open('Collection/'+i.split('/')[-1],'wb')
    fd.write(res.content)
    fd.close()
    
    loc.append('Collection/'+i.split('/')[-1])

In [24]:
data['Location'] = loc

In [26]:
data.to_csv('ImageScraping_2.csv', index =False)

In [27]:
data

,Link,Tags,Likes,Comments,Location
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Atmosphere, Sky Sky, Clouds, Blue",196,55,Collection/clouds-7050884__480.jpg
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Hummingbird Ornithology,",76,20,Collection/bird-7117346__340.jpg
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainfall, Rainbow, Subtropical",282,106,Collection/sea-7039471__340.jpg
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Japan, Sakura Blossoms, Road, Cherry",42,11,Collection/cherry-blossoms-7110279__340.jpg
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Flower, Marguerite, Cape Plant",39,15,Collection/cape-marguerite-7121992__340.jpg
...,...,...,...,...,...
1898,https://cdn.pixabay.com/photo/2018/02/16/22/08...,"Gull Flying, Seagull, Bird, Wings,",627,143,Collection/bird-3158784__340.jpg
1899,https://cdn.pixabay.com/photo/2022/03/22/16/00...,"Sea, Beach Cyprus, Island, Paphos,",5,0,Collection/island-7085366__340.jpg
1900,https://cdn.pixabay.com/photo/2013/02/21/19/10...,"Sea, Clouds, Boat Horizon, Cloudy,",867,146,Collection/sea-84629__340.jpg
1901,https://cdn.pixabay.com/photo/2022/03/23/16/22...,"Spring Blossom, Flowers, Cherry",11,8,Collection/cherry-blossom-7087492__480.jpg
